# Data Prediction

#### Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import models
import numpy as np
from PIL import Image
import cv2
import imutils

#### Global Variables

In [2]:
bg = None
temp_image = 'temp.png'

### Resize Image

Used to resize the image given as input.


In [3]:
def resizeImage(imageName):
    basewidth = 100
    img = Image.open(imageName)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img.save(imageName)

## Running Average

In [4]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return
    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

## Segimentation

In [5]:
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(),
                                 cv2.RETR_EXTERNAL,
                                 cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

## Getting Predicted Class

In [6]:
def getPredictedClass():
    # read the image
    image = cv2.imread(temp_image)
    # convert to greyscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # change to numpy array
    gray_image = gray_image.reshape(89, 100, 1)
    # change to numpy array with shape (1, 89, 100, 1) so model can receive it
    gray_image = np.array([gray_image])
    # predict the image
    prediction = model.predict(gray_image)
    # return a numpy array with all values for layers
    return prediction

## Displaying Result

In [7]:
def showStatistics(prediction):
    gestures = ['fist', 'palm', 'swing', 'ok']
    n = len(gestures)
    x = 30
    y = 30
    height = (n+3)*y
    width = 500
    textImage = np.zeros((height, width, 3), np.uint8)
    for i in range(0, len(gestures)):
        cv2.putText(textImage, 
                    gestures[i] + ' : ' + f"{prediction[0][i]:.2f}" , 
                    (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (255, 255, 255),
                    2)
        y = y + 30
        
    predicted_gesture = gestures[np.argmax(prediction)]
    sum = 0.00
    for i in prediction[0]:
        sum += i
    confidence = (np.amax(prediction) /  sum) * 100
    
    cv2.putText(textImage, 
            "Gesture: " + predicted_gesture, 
            (x, y),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 255),
            2)
    y += 30
    cv2.putText(textImage, 
            "Confidence: " + str(confidence) + "%", 
            (x, y),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 255),
            2)
    cv2.imshow("Statistics ", textImage)

#### (BUG) GPU memory overflow
> this is a workaround

In [8]:
#

method1 = False

if method1:
    import os
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    tf.compat.v1.reset_default_graph()
    gpu_options.allow_growth = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#

method2 = False

if method2:
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
    tf.compat.v1.keras.backend.set_session(
        tf.compat.v1.Session(config=config))

# 

method3 = False

if method3:
    physical_devices = tf.config.list_physical_devices('GPU')
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

## Loading Model

In [9]:
tf.compat.v1.reset_default_graph()
model = models.load_model('./TrainedModel/')

2021-12-18 09:17:59.797799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 09:17:59.874956: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 09:17:59.875137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 09:17:59.876584: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

## Predicting

In [29]:
# initialize weight for running average
aWeight = 0.5

# region of interest (ROI) coordinates
top, right, bottom, left = 10, 350, 225, 590

# initialize num of frames
num_frames = 0
start_recording = False


In [30]:
# get the reference to the webcam
camera = cv2.VideoCapture(0)

Press `s` to start recording, and press `q` to quit.
Please wait for some time till black background is formed.

In case you get camera not found error, intialize the camera again.

In [31]:
# keep looping, until interrupted
while(True):
    # get the current frame
    grabbed, frame = camera.read()

    if grabbed:
        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(
                    clone, [segmented + (right, top)], -1, (0, 0, 255))
                if start_recording:
                    cv2.imwrite(temp_image, thresholded)
                    resizeImage(temp_image)
                    # predictedClass, confidence = getPredictedClass()
                    prediction = getPredictedClass()
                    showStatistics(prediction)

                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

        if keypress == ord("s"):
            start_recording = True
    else:
        print("Error, Please check your camera")
        print(camera)
        break

# relaease the resources
camera.release()
cv2.destroyAllWindows()


In [19]:
# remove temporary image file
import os
os.remove(temp_image)